In [1]:
using ColoringNames
using JLD
using SwiftObjectStores

In [2]:
const cldata = load_monroe_data(dev_as_train=true, dev_as_test=false)
#Don't bother to load the actual training data, this is just the final test

ColoringNames.ColorDatasets{MLLabelUtils.LabelEnc.NativeLabels{SubString{String},311},ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}}(MLLabelUtils.LabelEnc.NativeLabels{SubString{String},311}(SubString{String}["acid","green","adobe","algae","almost","black","amber","amethyst","apple","apricot"  …  "vivid","vomit","warm","watermelon","weird","wheat","wine","wintergreen","wisteria","yuck"],Dict("russet"=>255,"celery"=>86,"lipstick"=>199,"dull"=>139,"chocolate"=>93,"sepia"=>263,"fern"=>152,"sickly"=>267,"y"=>44,"heliotrope"=>173…)),ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}(Any["acid green","acid green","acid green","acid green","acid green","acid green","acid green","acid green","acid green","acid green"  …  "yuck","yuck","yuck","yuck","yuck","yuck","yuck","yuck","yuck","yuck"],[1 1 … 311 311; 2 2 … 0 0; 0 0 … 0 0; 0 0 … 0 0],Float32[0.328092 0.665272 0.937255; 0.241007 0.712821 0.764706; … ; 0.0540541 0.308333 0.941176; 0.125874 0.

"/mnt_volume/julia_dir/v0.5/ColoringNames/expr/TestSet Eval.ipynb"

In [39]:
"Does some basic book keeping, and "
function update_storage!(params)
    params["final_testfile"] = joinpath(pwd(), "TestSet Eval.ipynb")
    params["final_git_hash"] =  strip(readstring(`git rev-parse --verify HEAD`))
    if haskey(params, "training_costs_o")
        delete!(params, "training_costs_o") #This uses up so much unnesc memory.
    end
    
    #Consistancy in naming
    if haskey(params, "mdl")
        #consistancy in naming
        params["model"] = params["mdl"]
        delete!(params, "mdl")
    end
    
    if haskey(params, "validation_set_results")
        #consistancy in naming
        params["dev_set_results"] = params["validation_set_results"]
        delete!(params, "validation_set_results")
    end
    params
end

update_storage!

# Full Task

In [25]:
1+1

2

In [49]:
for filename in [
        "../models/good/1706031148_sib0.5/params.jld",
        "../models/highdim/sib0.5/params.jld"
    ]
    params = load(filename)
    mdl = restore(TermToColorDistributionNetwork, filename)
    update_storage!(params)
    
    params["test_set_results"] = evaluate(mdl, cldata.test.terms_padded, cldata.test.colors)
    save(joinpath(dirname(filename),"final_"*basename(filename)), params)
end


In [50]:
for (filename) in [ 
    "../models/noml/sib0.5_or64/emprical_model.jld",
    "../models/noml/sib0.5_or64/smoothed_emprical_model.jld",
    "../models/noml/sib0.5_or256/emprical_model.jld",
    "../models/noml/sib0.5_or256/smoothed_emprical_model.jld"
    ]
    params = load(filename)
    update_storage!(params)
    mdl = params["model"]
        
    params["test_set_results"] = evaluate(mdl, cldata.test.texts, cldata.test.colors)
    save(joinpath(dirname(filename),"final_"*basename(filename)), params)
end
 


# Extrapolation

In [45]:
extrapo_texts = load("../models/good/extrapolate_sib0.5/params.jld", "eval_texts") 

extrapo_cldata = extrapolation_dataset(cldata, extrapo_texts)

LoadError: UndefVarError: extrapo_params not defined

In [54]:
for filename in [
        #"../models/good/extrapolate_sib0.5/params.jld",
        #"../models/good/1706031148_sib0.5/params.jld",
        "../models/highdim/extrapolate_sib0.5/params.jld"
        #"../models/highdim/sib0.5/params.jld"
    ]
    params = load(filename)
    mdl = restore(TermToColorDistributionNetwork, filename)
    update_storage!(params)
    delete!(params, "dev_set_results") #Dev set results may or may not have been on extrapolation data set, delete them
    params["test_set_results"] = evaluate(mdl, extrapo_cldata.test.terms_padded, extrapo_cldata.test.colors)
    save(joinpath(dirname(filename),"final_extrapolation_"*basename(filename)), params)
end

In [53]:
for filename in [
        "../models/noml/sib0.5_or64/final_smoothed_emprical_model.jld",
        "../models/noml/sib0.5_or256/final_smoothed_emprical_model.jld",
    ]
    params = load(filename)
    update_storage!(params)
    mdl = params["model"]
    delete!(params, "dev_set_results") #Dev set results may or may not have been on extrapolation data set, delete them
    params["test_set_results"] = evaluate(mdl, extrapo_cldata.test.texts, extrapo_cldata.test.colors)
    save(joinpath(dirname(filename),"final_extrapolation_"*basename(filename)), params)
end